In [260]:
import os
import librosa
import _pickle as cPickle
from scipy.io.wavfile import read
from pathlib import Path
import warnings
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
from sklearn import metrics
from keras.utils import to_categorical

In [261]:
train_file ="..\\data\\f_list\\wav_list_train.txt"    
test_file ="..\\data\\f_list\\wav_list_test.txt" 
def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""

    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines 
    delta to make it 40 dim feature vector"""    
    
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True)
    
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta)) 
    return combined
def get_features(file_path): 
    file_paths = open(file_path,'r')
    features_normalize = []
    total_feature = []
    features = np.asarray(())
    for path in file_paths:    
        path = path.strip()
        file_name=os.path.basename(path)
        audio, sr1 = librosa.load(path, sr=44100)
        data=librosa.resample(audio, sr1, 16000)
        vector   = extract_features(data,16000)
        if features.size == 0:
            features = vector
       
        else:
            features = np.vstack((features, vector)) 

        total_feature.append(features)
    return total_feature

In [262]:
def extract_label(file_path): 
    file_paths = open(file_path,'r')
    features_normalize = []
    total_feature = []
    labels = []
    features = np.asarray(())
    for path in file_paths:    
        path = path.strip()
        file_name=os.path.basename(path)
        audio, sr1 = librosa.load(path, sr=44100)
        data=librosa.resample(audio, sr1, 16000)
        vector   = extract_features(data,16000)

        if features.size == 0:
            features = vector
        else:
            features = np.vstack((features, vector))
        file_name=os.path.basename(path)
        split_filename=file_name.split('_')
        molecule_name=split_filename[0]
        for r in range(features.shape[0]):
            labels.append(molecule_name)
    return labels



In [263]:
from sklearn.preprocessing import LabelEncoder
# Extract features from the training data 
extracted_features_train = get_features(train_file) 
final_features_train = np.vstack(extracted_features_train)
# Extract features from the testing data 
extracted_features_test = get_features(test_file) 
final_features_test = np.vstack(extracted_features_test) 
# Extract labels from the training data 
train_label_value = extract_label(train_file)
train_final_label = np.array(train_label_value)
#print(train_label_value)
# Extract labels from the testing data 
test_label_value = extract_label(test_file)
test_final_label = np.array(test_label_value)
#print(test_final_label)
#print(test_final_label.shape)
x_train = final_features_train
x_test = final_features_test
#print(x_train)
#print(x_test)
y_train = train_final_label 
y_test =  test_final_label
#print(y_train)
#print(y_test)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test  = label_encoder.fit_transform(y_test)
#print(y_train)
#print(y_test)
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
#print(y_train_hot)
#print(y_test_hot)

In [264]:
import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(16, input_dim=40,activation='sigmoid'))
model.add(Dense(12, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy']) 
history=model.fit(x_train, y_train_hot, batch_size=1000, epochs=2500, verbose=1, validation_data=(x_test, y_test_hot))

Train on 23335 samples, validate on 23335 samples
Epoch 1/2500
23335/23335 [==============================] - 0s 19us/step - loss: 0.6720 - accuracy: 0.6087 - val_loss: 0.6495 - val_accuracy: 0.6582
Epoch 2/2500
23335/23335 [==============================] - 0s 13us/step - loss: 0.6453 - accuracy: 0.6582 - val_loss: 0.6429 - val_accuracy: 0.6582
Epoch 3/2500
23335/23335 [==============================] - 0s 10us/step - loss: 0.6425 - accuracy: 0.6582 - val_loss: 0.6423 - val_accuracy: 0.6582
Epoch 4/2500
23335/23335 [==============================] - 0s 10us/step - loss: 0.6423 - accuracy: 0.6582 - val_loss: 0.6422 - val_accuracy: 0.6582
Epoch 5/2500
23335/23335 [==============================] - 0s 10us/step - loss: 0.6423 - accuracy: 0.6582 - val_loss: 0.6422 - val_accuracy: 0.6582
Epoch 6/2500
23335/23335 [==============================] - 0s 11us/step - loss: 0.6422 - accuracy: 0.6582 - val_loss: 0.6421 - val_accuracy: 0.6582
Epoch 7/2500
23335/23335 [==============================

Epoch 110/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.5310 - accuracy: 0.6549 - val_loss: 0.5306 - val_accuracy: 0.6573
Epoch 111/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.5306 - accuracy: 0.6567 - val_loss: 0.5302 - val_accuracy: 0.6573
Epoch 112/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.5302 - accuracy: 0.6572 - val_loss: 0.5298 - val_accuracy: 0.6573
Epoch 113/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.5300 - accuracy: 0.6555 - val_loss: 0.5295 - val_accuracy: 0.6573
Epoch 114/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.5296 - accuracy: 0.6579 - val_loss: 0.5292 - val_accuracy: 0.6573
Epoch 115/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.5293 - accuracy: 0.6545 - val_loss: 0.5289 - val_accuracy: 0.6573
Epoch 116/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.5289 - accuracy:

23335/23335 [==============================] - 0s 9us/step - loss: 0.5107 - accuracy: 0.6583 - val_loss: 0.5103 - val_accuracy: 0.6576
Epoch 219/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.5106 - accuracy: 0.6565 - val_loss: 0.5103 - val_accuracy: 0.6575
Epoch 220/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.5102 - accuracy: 0.6601 - val_loss: 0.5110 - val_accuracy: 0.6573
Epoch 221/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.5104 - accuracy: 0.6615 - val_loss: 0.5107 - val_accuracy: 0.6573
Epoch 222/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.5102 - accuracy: 0.6599 - val_loss: 0.5107 - val_accuracy: 0.6578
Epoch 223/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.5103 - accuracy: 0.6558 - val_loss: 0.5097 - val_accuracy: 0.6576
Epoch 224/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.5101 - accuracy: 0.6575 - val_l

23335/23335 [==============================] - 0s 8us/step - loss: 0.5019 - accuracy: 0.6585 - val_loss: 0.5020 - val_accuracy: 0.6577
Epoch 327/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.5020 - accuracy: 0.6543 - val_loss: 0.5014 - val_accuracy: 0.6576
Epoch 328/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.5017 - accuracy: 0.6577 - val_loss: 0.5016 - val_accuracy: 0.6576
Epoch 329/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.5017 - accuracy: 0.6573 - val_loss: 0.5017 - val_accuracy: 0.6574
Epoch 330/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.5016 - accuracy: 0.6574 - val_loss: 0.5014 - val_accuracy: 0.6575
Epoch 331/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.5017 - accuracy: 0.6572 - val_loss: 0.5013 - val_accuracy: 0.6577
Epoch 332/2500
23335/23335 [==============================] - 0s 10us/step - loss: 0.5015 - accuracy: 0.6558 - val_

23335/23335 [==============================] - 0s 8us/step - loss: 0.4968 - accuracy: 0.6573 - val_loss: 0.4965 - val_accuracy: 0.6579
Epoch 435/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4966 - accuracy: 0.6575 - val_loss: 0.4973 - val_accuracy: 0.6574
Epoch 436/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4967 - accuracy: 0.6573 - val_loss: 0.4965 - val_accuracy: 0.6579
Epoch 437/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4966 - accuracy: 0.6590 - val_loss: 0.4964 - val_accuracy: 0.6579
Epoch 438/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4966 - accuracy: 0.6584 - val_loss: 0.4968 - val_accuracy: 0.6576
Epoch 439/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.4967 - accuracy: 0.6562 - val_loss: 0.4962 - val_accuracy: 0.6579
Epoch 440/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4965 - accuracy: 0.6572 - val_l

...

23335/23335 [==============================] - 0s 8us/step - loss: 0.4808 - accuracy: 0.6542 - val_loss: 0.4806 - val_accuracy: 0.6580
Epoch 2055/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.4808 - accuracy: 0.6563 - val_loss: 0.4806 - val_accuracy: 0.6580
Epoch 2056/2500
23335/23335 [==============================] - 0s 12us/step - loss: 0.4809 - accuracy: 0.6596 - val_loss: 0.4807 - val_accuracy: 0.6580
Epoch 2057/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.4809 - accuracy: 0.6572 - val_loss: 0.4806 - val_accuracy: 0.6579
Epoch 2058/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.4809 - accuracy: 0.6566 - val_loss: 0.4806 - val_accuracy: 0.6580
Epoch 2059/2500
23335/23335 [==============================] - 0s 11us/step - loss: 0.4809 - accuracy: 0.6580 - val_loss: 0.4806 - val_accuracy: 0.6579
Epoch 2060/2500
23335/23335 [==============================] - 0s 10us/step - loss: 0.4809 - accuracy: 0.656

23335/23335 [==============================] - 0s 9us/step - loss: 0.4807 - accuracy: 0.6585 - val_loss: 0.4804 - val_accuracy: 0.6579
Epoch 2109/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.4808 - accuracy: 0.6562 - val_loss: 0.4804 - val_accuracy: 0.6579
Epoch 2110/2500
23335/23335 [==============================] - 0s 10us/step - loss: 0.4807 - accuracy: 0.6606 - val_loss: 0.4805 - val_accuracy: 0.6579
Epoch 2111/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.4807 - accuracy: 0.6563 - val_loss: 0.4804 - val_accuracy: 0.6579
Epoch 2112/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.4808 - accuracy: 0.6585 - val_loss: 0.4804 - val_accuracy: 0.6579
Epoch 2113/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4808 - accuracy: 0.6571 - val_loss: 0.4804 - val_accuracy: 0.6579
Epoch 2114/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4807 - accuracy: 0.6556 

23335/23335 [==============================] - 0s 8us/step - loss: 0.4805 - accuracy: 0.6529 - val_loss: 0.4802 - val_accuracy: 0.6580
Epoch 2217/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.4805 - accuracy: 0.6561 - val_loss: 0.4801 - val_accuracy: 0.6579
Epoch 2218/2500
23335/23335 [==============================] - 0s 11us/step - loss: 0.4804 - accuracy: 0.6576 - val_loss: 0.4802 - val_accuracy: 0.6579
Epoch 2219/2500
23335/23335 [==============================] - 0s 12us/step - loss: 0.4804 - accuracy: 0.6578 - val_loss: 0.4801 - val_accuracy: 0.6579
Epoch 2220/2500
23335/23335 [==============================] - 0s 11us/step - loss: 0.4805 - accuracy: 0.6550 - val_loss: 0.4802 - val_accuracy: 0.6580
Epoch 2221/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4804 - accuracy: 0.6574 - val_loss: 0.4803 - val_accuracy: 0.6580
Epoch 2222/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4805 - accuracy: 0.656

23335/23335 [==============================] - 0s 8us/step - loss: 0.4802 - accuracy: 0.6588 - val_loss: 0.4799 - val_accuracy: 0.6579
Epoch 2325/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.4801 - accuracy: 0.6558 - val_loss: 0.4798 - val_accuracy: 0.6579
Epoch 2326/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4802 - accuracy: 0.6561 - val_loss: 0.4798 - val_accuracy: 0.6579
Epoch 2327/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4802 - accuracy: 0.6558 - val_loss: 0.4801 - val_accuracy: 0.6579
Epoch 2328/2500
23335/23335 [==============================] - 0s 10us/step - loss: 0.4802 - accuracy: 0.6546 - val_loss: 0.4801 - val_accuracy: 0.6579
Epoch 2329/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4802 - accuracy: 0.6561 - val_loss: 0.4798 - val_accuracy: 0.6579
Epoch 2330/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4802 - accuracy: 0.6556 

23335/23335 [==============================] - 0s 8us/step - loss: 0.4799 - accuracy: 0.6569 - val_loss: 0.4799 - val_accuracy: 0.6579
Epoch 2433/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.4800 - accuracy: 0.6560 - val_loss: 0.4797 - val_accuracy: 0.6580
Epoch 2434/2500
23335/23335 [==============================] - 0s 8us/step - loss: 0.4800 - accuracy: 0.6556 - val_loss: 0.4796 - val_accuracy: 0.6579
Epoch 2435/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.4799 - accuracy: 0.6555 - val_loss: 0.4796 - val_accuracy: 0.6579
Epoch 2436/2500
23335/23335 [==============================] - 0s 7us/step - loss: 0.4799 - accuracy: 0.6597 - val_loss: 0.4796 - val_accuracy: 0.6579
Epoch 2437/2500
23335/23335 [==============================] - 0s 10us/step - loss: 0.4799 - accuracy: 0.6557 - val_loss: 0.4796 - val_accuracy: 0.6579
Epoch 2438/2500
23335/23335 [==============================] - 0s 9us/step - loss: 0.4799 - accuracy: 0.6558 

In [265]:
y_pred = model.predict(x_test)
#print(y_pred)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#print(pred)
y = list()
for i in range(len(y_test_hot)):
    y.append(np.argmax(y_test_hot[i]))
#print(y)
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,y)
print('Accuracy is:', a*100)

Accuracy is: 65.79387186629526
